# Baltimore Tax Credit Project

## Requirements

Pandas and geopandas is needed to pull in the data and make modifications.

If you are trying to do this from a machine running Windows, `geopandas` can be difficult to get installed.  I recommend following [this guide](http://geoffboeing.com/2014/09/using-geopandas-windows/) by Geoff Boeing. That guide assumes you already have [Anaconda](http://continuum.io/downloads) installed, which is recommended anyway.

## Overview

### Working with the data

### Visualization

Visualization will be done with **Mapbox** through **Tilemill**.

## Imports and filenames

In [18]:
%matplotlib inline

import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) # This ensures we can view all the columns
pd.set_option('display.max_rows', None) # Force display of all requested rows

data_dir = "./data/"
property_zip = "Real_Property.zip"
property_shp = "Real_Property.shp"
output_file = "owner_occ_property.geojson"

## Load Data

Data is stored right now in `./data/Real_Property.zip`.

### Unzip data

In [7]:
import zipfile

zfile = zipfile.ZipFile(data_dir + property_zip)
zfile.extractall(data_dir)

### Load shapefile into memory

In [2]:
parcels = gpd.GeoDataFrame.from_file(data_dir + property_shp)
parcels.head()

,ARTAXBAS,AR_OWNER,ASSESGRP,ASSESSOR,BFCVIMPR,BFCVLAND,BLDGSQFT,BLDG_NO,BLOCK,BLOCKLOT,CCREDAMT,CITYCRED,CITY_TAX,CURRIMPR,CURRLAND,DEEDBOOK,DEEDPAGE,DHCDUSE1,DHCDUSE2,DHCDUSE3,DHCDUSE4,DISTSWCH,DIST_ID,DWELUNIT,EFF_UNIT,EXMPCODE,EXMPIMPR,EXMPLAND,EXMPTYPE,EXTD_ZIP,FRACTION,FULLADDR,FULLCASH,GRNDRENT,IMPREXMP,LANDEXMP,LDATE,LOT,LOT_SIZE,NEIGHBOR,NO_IMPRV,OBJECTID,OWNER_1,OWNER_2,OWNER_3,OWNER_ABBR,OWNMDE,PERMHOME,PIN,PROPDESC,RESPAGCY,ROOMUNIT,RPDELTAG,SALEDATE,SALEPRIC,SCREDAMT,SDATCODE,SPANFRAC,SPAN_NUM,SRVCCNTR,STATCRED,STATETAX,STDIRPRE,ST_NAME,ST_TYPE,SUBTYPE_GE,ShapeSTAre,ShapeSTLen,TAXBASE,UNIT_NUM,USEGROUP,WARD,YEAR_BUILD,ZIP_CODE,ZONECODE,created_da,created_us,geometry,last_edi_1,last_edite
0,70200,N,2,310,60000,30000,1920,1923,3941A,3941A012,0.00,0,1578.10,56200,14000,FMC12906,0181,1123,0000,0000,0000,None,None,2,0,00,0,0,None,3106,None,1923 E 32ND ST,70200,0,0,0,07022015,012,24-3X131,COLDSTREAM HOMESTEAD MONTEBELLO,None,1132760,"DOMINION RENTALS, LLC",None,None,None,F,N,3941A012,None,None,0,None,09032010,26000,0,11130,None,0,4,0,78.62,E,32ND,ST,1,3186.729126,310.665602,70200,None,R,09,1924,21218,R-6,2015-07-06T17:31:31.000Z,EGISDATA,POLYGON ((-76.58816450001287 39.32685429146552...,2015-07-06T17:31:31.000Z,EGISDATA
1,10200,N,2,384,20000,5000,0,745,1628,1628 023,0.00,0,229.30,8200,2000,FMC13310,0379,1123,6814,0000,0000,None,None,2,0,00,0,0,None,2630,None,745 N KENWOOD AVE,10200,84,0,0,07022015,023,15X70,MADISON-EASTEND,None,1132642,"THOMAS, JOHN",None,None,None,L,N,1628023,None,None,0,None,02222011,10000,0,11440,None,0,3,0,11.42,N,KENWOOD,AVE,1,996.647705,168.483739,10200,None,R,07,1915,21205,R-8,2015-07-06T17:31:31.000Z,EGISDATA,POLYGON ((-76.57775070669615 39.29995842106153...,2015-07-06T17:31:31.000Z,EGISDATA
2,147700,H,1,320,99700,58300,1287,6003,5695E,5695E039,0.00,0,3320.30,87700,60000,FMC15496,0164,1111,0000,0000,0000,None,None,1,0,00,0,0,None,2525,None,6003 GLENFALLS AVE,147700,0,0,0,07022015,039,54X90-1,GLENHAM-BELHAR,None,1132761,"SPENCER, CAROLYN M",None,None,None,F,H,5695E039,None,None,0,None,08062013,92000,0,11110,None,0,4,0,165.42,None,GLENFALLS,AVE,1,4856.770325,287.900128,147700,None,R,27,1951,21206,R-5,2015-07-06T17:31:31.000Z,EGISDATA,POLYGON ((-76.54108672842762 39.34814590666414...,2015-07-06T17:31:31.000Z,EGISDATA
3,228933,N,3,310,120600,80000,2652,908,1880,1880 005,0.00,0,5146.41,185600,100000,FMC12281,0224,1123,0000,0000,0000,None,None,2,0,00,0,0,None,4947,None,908 S ELLWOOD AVE,285600,0,0,0,07022015,005,18X100,CANTON,None,1132643,"KNOTT, BRANNAN H",None,None,None,F,N,1880005,None,None,0,None,12182009,258000,0,11230,None,0,2,0,256.40,S,ELLWOOD,AVE,1,1791.297729,235.567251,228933,None,R,01,1910,21224,R-8,2015-07-06T17:31:31.000Z,EGISDATA,POLYGON ((-76.57277386208826 39.28202638475474...,2015-07-06T17:31:31.000Z,EGISDATA
4,142900,H,1,305,95200,77000,1448,3004,4455,4455 038,876.72,0,3212.39,80900,62000,FMC05537,0292,1111,0000,0000,0000,None,None,1,0,00,0,0,None,4012,None,3004 ROCKWOOD AVE,142900,0,0,0,07022015,038,49X144,GLEN,None,1132762,"WILLIAMS, CURTIS",None,None,None,F,H,4455038,None,None,0,None,06032004,87000,0,11110,None,0,6,0,160.05,None,ROCKWOOD,AVE,1,7034.895325,385.790800,142900,None,R,27,1929,21215,R-4,2015-07-06T17:31:31.000Z,EGISDATA,POLYGON ((-76.68004942223392 39.35893934895754...,2015-07-06T17:31:31.000Z,EGISDATA


## Filter Original Data

To make the data easier to work with, going to create a filtered version of the dataset. In particular I want to only keep columns that I think will be useful to have as part of the map.

It seems like the columns are as follows (this is my best guess, verified when possible):

| Col Name | Desc |
|----------|------|
| **geometry** | Physical location of parcel boundaries (important) |
| | |
| **AR_OWNER** | Owner occupied? `H` = Yes, `N` = No |
| | |
| **BLOCK** | Block number |
| **LOT** | Lot number |
| **WARD** | Ward number |
| **BLOCKLOT** | Combined block and lot fields |
| | |
| **FULLADDR** | Full street address of property |
| **NEIGHBOR** | Neighborhood |
| | |
| **CITY_TAX** | City tax amount |
| **STATETAX** | State tax amount |
| **TAXBASE** | Total taxable value of property |
| | |
| **CCREDAMT** | City homestead tax credit amount |
| **SCREDAMT** | State homestead tax credit amount |
| **CITYCRED** | Not sure |
| **STATCRED** | State homeowner's tax credit amount |
| | |
| **OWNMDE** | Not sure, has values `F`, `L`, or `None`.  |
| **PERMHOME** | Owner occupied? `H` = Yes, `N` = No. May be identical to **AR_OWNER** |
| **SALEDATE** | Date of most recent sale of property |

Below I'll keep a subset of these as defined in `col_list`.

**Note:** The geometry for each property is stored in `geometry` so we definitely need to keep this field in order to be able to display the parcels on the map.

In [44]:
col_list = ["BLOCK", "LOT", "WARD", "BLOCKLOT", 
            "FULLADDR", "NEIGHBOR", 
            "CITY_TAX", "STATETAX", "TAXBASE",
            "CCREDAMT", "SCREDAMT", "CITYCRED", "STATCRED", 
            "PERMHOME", "SALEDATE"]

filt_parcels = gpd.GeoDataFrame()

filt_parcels["geometry"] = parcels["geometry"]

for col_name in col_list:
    filt_parcels[col_name] = parcels[col_name]
    
filt_parcels.head(n = 20)


,geometry,BLOCK,LOT,WARD,BLOCKLOT,FULLADDR,NEIGHBOR,CITY_TAX,STATETAX,TAXBASE,CCREDAMT,SCREDAMT,CITYCRED,STATCRED,PERMHOME,SALEDATE
0,POLYGON ((-76.58816450001287 39.32685429146552...,3941A,012,09,3941A012,1923 E 32ND ST,COLDSTREAM HOMESTEAD MONTEBELLO,1578.10,78.62,70200,0.00,0,0,0,N,09032010
1,POLYGON ((-76.57775070669615 39.29995842106153...,1628,023,07,1628 023,745 N KENWOOD AVE,MADISON-EASTEND,229.30,11.42,10200,0.00,0,0,0,N,02222011
2,POLYGON ((-76.54108672842762 39.34814590666414...,5695E,039,27,5695E039,6003 GLENFALLS AVE,GLENHAM-BELHAR,3320.30,165.42,147700,0.00,0,0,0,H,08062013
3,POLYGON ((-76.57277386208826 39.28202638475474...,1880,005,01,1880 005,908 S ELLWOOD AVE,CANTON,5146.41,256.40,228933,0.00,0,0,0,N,12182009
4,POLYGON ((-76.68004942223392 39.35893934895754...,4455,038,27,4455 038,3004 ROCKWOOD AVE,GLEN,3212.39,160.05,142900,876.72,0,0,0,H,06032004
5,POLYGON ((-76.58703107337121 39.37025410730454...,5210F,019,27,5210F019,1336 GITTINGS AVE,IDLEWOOD,2675.12,133.28,119000,0.00,0,0,0,H,07232008
6,POLYGON ((-76.60312792601935 39.32580693932493...,4096,029,09,4096 029,943 GORSUCH AVE,BETTER WAVERLY,0.00,0.00,0,0.00,0,0,0,N,08021999
7,POLYGON ((-76.59525309668561 39.31528933945137...,4165,030,08,4165 030,1612 CLIFTVIEW AVE,DARLEY PARK,330.46,16.46,14700,0.00,0,0,0,H,06172011
8,POLYGON ((-76.61786605467449 39.31942610479524...,3637,053,12,3637 053,4 W 26TH ST,CHARLES VILLAGE,3166.69,157.77,140867,0.00,0,0,0,N,07301991
9,"POLYGON ((-76.6340391486756 39.31211664855822,...",3427,039,13,3427 039,2064 LINDEN AVE,RESERVOIR HILL,4142.32,206.38,184267,0.00,0,0,0,N,05211996


### Check Filtered Data

Here I'm going to verify that we can identify properties that are owner occupied.


In [57]:
owned_parcels = filt_parcels[filt_parcels["PERMHOME"] == "H"]
owned_parcels.reset_index(drop=True, inplace=True)
owned_parcels.head(n = 5)

,geometry,BLOCK,LOT,WARD,BLOCKLOT,FULLADDR,NEIGHBOR,CITY_TAX,STATETAX,TAXBASE,CCREDAMT,SCREDAMT,CITYCRED,STATCRED,PERMHOME,SALEDATE
0,POLYGON ((-76.54108672842762 39.34814590666414...,5695E,039,27,5695E039,6003 GLENFALLS AVE,GLENHAM-BELHAR,3320.30,165.42,147700,0.00,0,0,0,H,08062013
1,POLYGON ((-76.68004942223392 39.35893934895754...,4455,038,27,4455 038,3004 ROCKWOOD AVE,GLEN,3212.39,160.05,142900,876.72,0,0,0,H,06032004
2,POLYGON ((-76.58703107337121 39.37025410730454...,5210F,019,27,5210F019,1336 GITTINGS AVE,IDLEWOOD,2675.12,133.28,119000,0.00,0,0,0,H,07232008
3,POLYGON ((-76.59525309668561 39.31528933945137...,4165,030,08,4165 030,1612 CLIFTVIEW AVE,DARLEY PARK,330.46,16.46,14700,0.00,0,0,0,H,06172011
4,POLYGON ((-76.56944852517457 39.35443981507301...,5370,032,27,5370 032,5311 GRINDON AVE,HAMILTON HILLS,2852.71,142.13,126900,232.26,0,0,0,H,02282002


In [39]:
print("Identified owner occupied properties represent {:.1f}% of the original dataset".format(100.0 * len(owned_parcels)/len(parcels)))

Identified owner occupied properties represent 48.3% of the original dataset


#### Comparison with US Census
About 50% of the homes in the dataset are classified as owner occupied.  As a sanity check, we will compare this number to those reported by the US Census.  A [2013 US Census Report](http://www2.census.gov/programs-surveys/ahs/2013/factsheets/ahs13-3_Baltimore.pdf) of the Baltimore metropolitan area puts the number closer to 60%, but is considering a larger population than the city alone. This [Census Quick Facts Report](http://quickfacts.census.gov/qfd/states/24/24510.html) puts the homeownership rate at 48.3%, so presumably we have correctly identified these properties.

## Prepare for Export

Before exporting the data we want to create any additional fields we might need and eliminate fields we don't need.  

Useful fields to add:
* Binary field indicating whether or not an owner has applied for the credit
* Combine city tax and state tax into a single total tax field
* Extract year from sale date

Fields to drop:
* `CITY_TAX`
* `STATETAX`
* `SALEDATE`

In [58]:
# Add new fields
owned_parcels.loc[:,"HASCRED"] = pd.Series(owned_parcels["STATCRED"] > 0).astype(int)
owned_parcels.loc[:,"SALEYEAR"] = owned_parcels["SALEDATE"].map(lambda x: str(x)[-4:])
owned_parcels.loc[:,"TOTALTAX"] = pd.Series(owned_parcels["CITY_TAX"] + owned_parcels["STATETAX"])

# Delete unnecessary fields
owned_parcels = owned_parcels.drop(['CITY_TAX', 'STATETAX', 'SALEDATE'], axis=1)

# Display preview
owned_parcels.head(n=5)

,geometry,BLOCK,LOT,WARD,BLOCKLOT,FULLADDR,NEIGHBOR,TAXBASE,CCREDAMT,SCREDAMT,CITYCRED,STATCRED,PERMHOME,HASCRED,SALEYEAR,TOTALTAX
0,POLYGON ((-76.54108672842762 39.34814590666414...,5695E,039,27,5695E039,6003 GLENFALLS AVE,GLENHAM-BELHAR,147700,0.00,0,0,0,H,0,2013,3485.72
1,POLYGON ((-76.68004942223392 39.35893934895754...,4455,038,27,4455 038,3004 ROCKWOOD AVE,GLEN,142900,876.72,0,0,0,H,0,2004,3372.44
2,POLYGON ((-76.58703107337121 39.37025410730454...,5210F,019,27,5210F019,1336 GITTINGS AVE,IDLEWOOD,119000,0.00,0,0,0,H,0,2008,2808.40
3,POLYGON ((-76.59525309668561 39.31528933945137...,4165,030,08,4165 030,1612 CLIFTVIEW AVE,DARLEY PARK,14700,0.00,0,0,0,H,0,2011,346.92
4,POLYGON ((-76.56944852517457 39.35443981507301...,5370,032,27,5370 032,5311 GRINDON AVE,HAMILTON HILLS,126900,232.26,0,0,0,H,0,2002,2994.84


## Export Data

Now that we have extracted the records we are interested in along with all the relevant fields, we will export the data as a geojson file.

In [59]:
with open(data_dir + output_file,'w') as f:
    f.write(owned_parcels.to_json())